In [55]:
import pandas as pd
import numpy as np

from google.colab import drive
drive.mount('/content/drive')
!unzip 'drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip'

train=pd.read_csv('open/train.csv')
test=pd.read_csv('open/test.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archive:  drive/MyDrive/kubig_ML/235713_신용카드 사용자 연체 예측 AI 경진대회_data.zip
replace open/train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace open/sample_submission.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace open/test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


# 전처리 해본 것

In [56]:
#완전히 중복되는 데이터 삭제
train=train[~train.drop(columns=['credit','index','FLAG_MOBIL']).duplicated(keep='first')]

# day_employed 양수값 0으로 변환
train['DAYS_EMPLOYED'].loc[train['DAYS_EMPLOYED']>0]=0

# occyp_type No job 및 Laborers로 채우기

train['occyp_type'].loc[(train['DAYS_EMPLOYED']==0)&(train['occyp_type'].isna())]='No job'
train['occyp_type']=train['occyp_type'].fillna('Laborers')


# 성별 변수 인코딩
train['gender']=train['gender'].replace(['M','F'],[1,0])
train['car']=train['car'].replace(['Y','N'],[1,0])
train['reality']=train['reality'].replace(['Y','N'],[1,0])

train.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# 자녀유무 변수
train['kids']=0
train['kids'] = np.where(train['child_num'] !=0, 1, 0)
train.drop(['child_num'],axis=1,inplace=True)


# 인코딩 > ordinalencoder로 바꿈
from sklearn.preprocessing import OrdinalEncoder
cat=train.dtypes[train.dtypes=="object"].index.tolist()
encoder = OrdinalEncoder()
train[cat] = encoder.fit_transform(train[cat],train['credit'])


# family_size outlier 삭제
train=train[(train['family_size']<7)]
train=train.reset_index(drop=True)


# income_total, family_size 스케일링
from sklearn.preprocessing import RobustScaler
scaler3 = RobustScaler()
scale=['income_total','family_size']
train[scale] = scaler3.fit_transform(train[scale])
test[scale] = scaler3.transform(test[scale])


/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
<ipython-input-56-a5c80586e61f>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train['occyp_type']=train['occyp_type'].fillna('Laborers')
<ipython-input-56-a5c80586e61f>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/sta

# Test 제출용

## test 전처리

In [57]:
# 'DAYS_EMPLOYED' 365243->0
test['DAYS_EMPLOYED'].loc[test['DAYS_EMPLOYED']>0]=0

# 'occyp_type' No job 추가, Laborers로 결측치 채우기
test['occyp_type'].loc[(test['DAYS_EMPLOYED']==0)&(test['occyp_type'].isna())]='No job'
test['occyp_type']=test['occyp_type'].fillna('Laborers')

# categorical 변수 ordinal encoding
test[cat]=encoder.transform(test[cat])

# 'gender','car','reality' 0,1로 encoding
test['gender']=test['gender'].replace(['M','F'],[1,0])
test['car']=test['car'].replace(['Y','N'],[1,0])
test['reality']=test['reality'].replace(['Y','N'],[1,0])

index=test['index'] # 나중에 index 쓸 것이라서 저장

#'FLAG_MOBIL','index' drop
test.drop(['index','FLAG_MOBIL'],axis=1,inplace=True)

# 자식 유무 변수 'kids' 추가 후 'child_num' drop
test['kids']=0
test['kids'] = np.where(test['child_num'] !=0, 1, 0)
test.drop(['child_num'],axis=1,inplace=True)



/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [58]:
# train data X,y 만들어주기
X_train=train.drop(columns='credit')
y_train=train['credit']

In [59]:
from lightgbm import LGBMClassifier
model_lgbc = LGBMClassifier(n_estimators=450, max_depth=10,
                            colsample_bytree=1, min_child_weight=3, subsample=0.001, 
                            random_state=2)
model_lgbc.fit(X_train, y_train)

y_pred=model_lgbc.predict_proba(test)

In [60]:
y_pred

array([[0.03558961, 0.08121589, 0.88319449],
       [0.25951836, 0.19281805, 0.54766359],
       [0.03998946, 0.03867854, 0.921332  ],
       ...,
       [0.02888461, 0.10373668, 0.8673787 ],
       [0.1630319 , 0.15581805, 0.68115005],
       [0.02850407, 0.2592793 , 0.71221663]])

In [62]:
y_pred_df=pd.DataFrame(y_pred)

In [63]:
y_pred_df=pd.concat([index,y_pred_df],axis=1)
y_pred_df.to_csv('base_submission.csv',index = False)